# Feature Selection

### Load the data from file and save train/test data

In [1]:
#code
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/extracted_data.csv')
df.dropna(axis=1, inplace=True)
df.replace([np.inf, -np.inf], np.nan).dropna(axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)

df.head()

,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""median""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""var""__maxlag_40","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Zone29_K1,Zone29_K2,Zone29_K3,Zone30_K1,Zone30_K2,Zone30_K3,Zone31_K1,Zone31_K2,Zone31_K3,target
0,8.439290e-08,0.000074,0.0,0.0,0.0,0.000013,-0.480982,-9.571038e-08,1.838974e-08,0.000014,...,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
1,8.337982e-08,0.000071,0.0,0.0,0.0,0.000013,-0.616363,-1.284958e-07,1.649928e-08,0.000015,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
2,1.032448e-07,0.000059,0.0,0.0,0.0,0.000012,-0.241002,-4.498370e-08,1.976525e-08,0.000014,...,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
3,9.590627e-08,0.000061,0.0,0.0,0.0,0.000013,-0.528130,-1.032466e-07,1.703194e-08,0.000015,...,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,1
4,9.772771e-08,0.000060,0.0,0.0,0.0,0.000013,-0.493841,-9.472176e-08,1.738857e-08,0.000014,...,8.000000e-11,9.000000e-11,3.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,1


In [2]:
#code
from sklearn.model_selection import train_test_split

seed = 0
df_train, df_test = train_test_split(df, test_size=0.3, random_state=seed)
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [3]:
df_train.to_csv('tmp/train_data.csv')
df_test.to_csv('tmp/test_data.csv')

### Read train data

In [1]:
import pandas as pd

df = pd.read_csv('tmp/train_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.drop('index', axis=1, inplace=True)

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [2]:
df.head()

,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""median""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""var""__maxlag_40","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Zone29_K1,Zone29_K2,Zone29_K3,Zone30_K1,Zone30_K2,Zone30_K3,Zone31_K1,Zone31_K2,Zone31_K3,target
0,9.217900e-08,0.000060,0.0,0.0,0.0,0.000014,-0.563318,-1.164333e-07,1.770878e-08,0.000015,...,8.000000e-11,9.000000e-11,3.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
1,1.034098e-07,0.000063,0.0,0.0,0.0,0.000012,-0.277973,-5.242509e-08,1.965012e-08,0.000014,...,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,1
2,9.362751e-08,0.000069,0.0,0.0,0.0,0.000014,-0.575139,-1.060775e-07,1.524125e-08,0.000015,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
3,1.093936e-07,0.000051,0.0,0.0,0.0,0.000013,-0.268240,-5.179699e-08,2.017696e-08,0.000014,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,1
4,9.681778e-08,0.000062,0.0,0.0,0.0,0.000013,-0.551382,-1.007458e-07,1.539783e-08,0.000014,...,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,1


### Automatic sccoring function

In [18]:
#code
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score

seed = 0

def automatic_scoring_tt(X_train, y_train, X_test, y_test):
    y_pred = RandomForestClassifier(n_estimators=100, random_state=seed).fit(X_train, y_train).predict(X_test)
    print("roc_auc: " + str(roc_auc_score(y_test, y_pred)))
    print("f1_score: " + str(f1_score(y_test, y_pred)))

def automatic_scoring_cv(X, y):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)
    average_score = cross_val_score(estimator=rf_model, X=X, y=y, cv=5, scoring='roc_auc').mean()
    return average_score

In [4]:
automatic_scoring_cv(X, y)

0.887680530658183

### 1. Feature selector that removes all low-variance features.

In [5]:
#code
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=(.8 * (1 - .8)))
selector.fit(X)
columns = X.columns[selector.get_support(indices=True)]
X = X[columns]

X.head()

,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_2,Flow rate__ar_coefficient__k_10__coeff_3,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""",...,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__sum_of_reoccurring_values,Zone9_Pressure__sum_values,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_1,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_2,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_3,Zone9_Pressure__value_count__value_0,Zone9_Pressure__variance
0,1.138229,-0.134713,-0.043323,0.001294,1.0,2.446854,481.0,468.0,46.464240,9.547049,...,0.0,0.0,7018882.57,0.0,1.0,1.899014e+11,3.748467e+11,5.577161e+11,717.0,2.450095e+08
1,1.083782,-0.159063,0.033663,-0.011084,8.0,1.961303,413.0,450.0,32.142801,12.080702,...,0.0,0.0,6659896.35,0.0,1.0,3.375301e+11,6.703761e+11,1.000059e+12,694.0,3.871354e+08
2,1.099869,-0.080847,-0.040066,-0.014425,1.0,2.612159,585.0,407.0,49.179479,9.310614,...,0.0,0.0,17866549.00,0.0,0.0,5.840482e+11,1.159956e+12,1.734019e+12,655.0,8.799311e+08
3,1.047635,-0.091263,0.032230,0.022891,1.0,1.107839,447.0,420.0,22.383663,17.811326,...,0.0,0.0,5865586.98,0.0,1.0,4.033242e+10,7.963747e+10,1.183241e+11,671.0,2.360060e+08
4,1.089493,-0.083334,-0.025769,0.005109,1.0,2.291503,417.0,574.0,46.155024,9.857893,...,0.0,0.0,5269945.00,1.0,1.0,2.359112e+11,4.571904e+11,6.741021e+11,750.0,1.717643e+08


In [6]:
automatic_scoring_cv(X, y)

0.884140442993701

### 2. Feature selector that checks correlation between 2 features. If correlation is high, 1 feature is removed.

In [8]:
#code
import numpy as np

def correl(data, threshold):
    corr = data.corr()
    print('Corr calculated')
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if columns[j]:
                    columns[j] = False
    selected_columns = data.columns[columns]
    data = data[selected_columns]
    return data

X = correl(X, 0.9)

X.head()

Corr calculated


,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""","Flow rate__fft_coefficient__coeff_10__attr_""angle""","Flow rate__fft_coefficient__coeff_11__attr_""angle""",...,Zone9_Pressure__partial_autocorrelation__lag_3,Zone9_Pressure__partial_autocorrelation__lag_9,Zone9_Pressure__quantile__q_0.7,Zone9_Pressure__range_count__max_1__min_-1,Zone9_Pressure__skewness,Zone9_Pressure__spkt_welch_density__coeff_2,Zone9_Pressure__spkt_welch_density__coeff_5,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002
0,1.138229,0.001294,1.0,2.446854,481.0,468.0,46.464240,9.547049,-120.433969,175.096636,...,-0.006129,-0.012070,0.000,717.0,1.903344,6.680968e+08,4.716441e+06,0.0,0.0,1.0
1,1.083782,-0.011084,8.0,1.961303,413.0,450.0,32.142801,12.080702,123.824497,-118.457850,...,-0.051759,-0.000288,0.000,694.0,2.486201,2.407189e+07,3.937312e+05,0.0,0.0,1.0
2,1.099869,-0.014425,1.0,2.612159,585.0,407.0,49.179479,9.310614,-53.836446,91.650082,...,-0.033722,-0.009004,6821.085,655.0,1.192949,1.712010e+09,4.552468e+07,0.0,0.0,0.0
3,1.047635,0.022891,1.0,1.107839,447.0,420.0,22.383663,17.811326,-115.869798,-81.600271,...,-0.096976,-0.048858,0.000,671.0,2.232394,8.624258e+07,5.631500e+05,0.0,0.0,1.0
4,1.089493,0.005109,1.0,2.291503,417.0,574.0,46.155024,9.857893,-105.880158,164.234708,...,0.002606,-0.010698,0.000,750.0,2.665094,3.478049e+07,2.810230e+04,0.0,1.0,1.0


In [9]:
automatic_scoring_cv(X, y)

0.8752469298893498

In [11]:
X.['target'] = y
X.to_csv('tmp/selected_data_1.csv')

### 3. Select features according to a percentile of the highest scores (mutual_info_classif)

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/selected_data_1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

df.head()

,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""","Flow rate__fft_coefficient__coeff_10__attr_""angle""","Flow rate__fft_coefficient__coeff_11__attr_""angle""",...,Zone9_Pressure__partial_autocorrelation__lag_9,Zone9_Pressure__quantile__q_0.7,Zone9_Pressure__range_count__max_1__min_-1,Zone9_Pressure__skewness,Zone9_Pressure__spkt_welch_density__coeff_2,Zone9_Pressure__spkt_welch_density__coeff_5,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002,target
0,1.138229,0.001294,1.0,2.446854,481.0,468.0,46.464240,9.547049,-120.433969,175.096636,...,-0.012070,0.000,717.0,1.903344,6.680968e+08,4.716441e+06,0.0,0.0,1.0,0
1,1.083782,-0.011084,8.0,1.961303,413.0,450.0,32.142801,12.080702,123.824497,-118.457850,...,-0.000288,0.000,694.0,2.486201,2.407189e+07,3.937312e+05,0.0,0.0,1.0,1
2,1.099869,-0.014425,1.0,2.612159,585.0,407.0,49.179479,9.310614,-53.836446,91.650082,...,-0.009004,6821.085,655.0,1.192949,1.712010e+09,4.552468e+07,0.0,0.0,0.0,0
3,1.047635,0.022891,1.0,1.107839,447.0,420.0,22.383663,17.811326,-115.869798,-81.600271,...,-0.048858,0.000,671.0,2.232394,8.624258e+07,5.631500e+05,0.0,0.0,1.0,1
4,1.089493,0.005109,1.0,2.291503,417.0,574.0,46.155024,9.857893,-105.880158,164.234708,...,-0.010698,0.000,750.0,2.665094,3.478049e+07,2.810230e+04,0.0,1.0,1.0,1


In [ ]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

selperc = SelectPercentile(mutual_info_classif, percentile=1)
X_features = selperc.fit_transform(X, y)
columns = np.asarray(X.columns.values)
support = np.asarray(selperc.get_support())
columns_with_support = columns[support]

X = X[columns_with_support]
X.head()

In [ ]:
automatic_scoring_cv(X, y)

In [ ]:
X.['target'] = y
X.to_csv('tmp/selected_data_2.csv')

### 4. Feature ranking with recursive feature elimination and cross-validated selection of the best number of features.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/selected_data_2.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

df.head()

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

sel = RFECV(SVC(kernel="linear"), step=1, cv=5, n_jobs=6)
sel = sel.fit(X, y)
columns = np.asarray(X.columns.values)
support = np.asarray(sel.get_support())
columns_with_support = columns[support]

In [ ]:
X[columns_with_support]

In [ ]:
automatic_scoring_cv(X[columns_with_support], y)

### 5. Test with test data

In [11]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('tmp/selected_data_2.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]

X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

df_train.head()

,"Flow rate__fft_coefficient__coeff_1__attr_""angle""",Flow rate__length,"Zone10_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone11_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Zone11_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",Zone15_Pressure__count_below_mean,"Zone15_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__fft_aggregated__aggtype_""kurtosis""",...,"Zone31_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone3_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone4_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone5_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone6_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Zone6_Pressure__fft_coefficient__coeff_2__attr_""angle""",Zone6_Pressure__partial_autocorrelation__lag_2,"Zone7_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone7_Pressure__fft_coefficient__coeff_2__attr_""angle""",target
0,-133.156136,949.0,-2.964372e+04,95.686572,59.248009,0.505982,45.611017,568.0,75.028473,5.776310,...,-9.310171e+05,148.489858,-156.469317,-139.481181,102.743651,-170.198348,-0.110227,1.712034,92.717040,0
1,-156.043635,863.0,-1.146507e+06,22.493810,41.514168,-1.069010,113.872216,566.0,50.880973,7.862577,...,-1.793086e+06,126.229099,129.473073,131.994737,22.769477,130.317641,-0.399107,-1.096303,98.660564,1
2,-129.957522,992.0,3.226080e+05,122.991554,56.670234,1.841951,75.079191,648.0,78.834497,5.784401,...,1.614355e+06,80.454668,61.397743,53.669202,127.463038,87.747322,-0.190496,1.056589,78.150238,0
3,-159.279799,867.0,-1.819657e+06,2.819112,37.574585,-1.185652,97.722058,539.0,43.575737,9.189750,...,-8.465515e+05,134.725121,141.749812,143.606770,5.531214,136.339948,-0.368715,-1.208694,99.725616,1
4,-133.498270,991.0,-1.776349e+05,98.944519,58.988249,-0.474574,82.765689,617.0,64.398653,6.897727,...,2.690433e+05,139.196540,130.846438,129.830750,72.038636,133.019809,-0.191307,-0.531102,110.545236,1


In [12]:
df_test = pd.read_csv('tmp/tsfresh/test_data.csv')
df_test = df_test[df_train.columns.values]

X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]

df_test.head()

,"Flow rate__fft_coefficient__coeff_1__attr_""angle""",Flow rate__length,"Zone10_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone11_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Zone11_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",Zone15_Pressure__count_below_mean,"Zone15_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__fft_aggregated__aggtype_""kurtosis""",...,"Zone31_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone3_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone4_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone5_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone6_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Zone6_Pressure__fft_coefficient__coeff_2__attr_""angle""",Zone6_Pressure__partial_autocorrelation__lag_2,"Zone7_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone7_Pressure__fft_coefficient__coeff_2__attr_""angle""",target
0,-132.657088,973.0,1.564942e+06,117.069535,46.376092,0.067698,53.182500,702.0,72.019185,6.131523,...,1.652083e+06,81.742192,74.563286,80.903365,133.200825,91.252110,-0.333313,0.046203,67.955545,1
1,-119.297751,973.0,1.424868e+06,106.692943,64.763855,3.213600,53.219041,665.0,71.394873,6.197314,...,2.570737e+05,93.858463,99.154413,109.544397,127.613402,112.082132,-0.040622,3.410851,85.090897,1
2,-141.162775,944.0,-1.943457e+06,96.021847,49.126875,-1.204123,83.269429,580.0,60.966327,7.196892,...,-1.103678e+06,167.698671,170.618512,173.922272,84.474400,175.844645,-0.290299,-1.183838,139.460673,0
3,-145.550702,951.0,5.123867e+05,98.584007,60.374352,-1.464118,115.421238,561.0,60.996693,7.075770,...,-3.557222e+06,131.727442,151.476200,145.624128,282.534301,147.382722,-0.083639,-1.407661,121.750780,0
4,-142.952705,984.0,-1.542463e+06,90.650340,56.286225,1.060877,104.030742,624.0,80.434059,5.587676,...,1.893550e+05,133.518869,163.207849,167.696316,128.337659,138.262600,-0.152938,2.653337,79.505136,1


In [19]:
automatic_scoring_tt(X_train, y_train, X_test, y_test)

roc_auc: 0.7762711864406779
f1_score: 0.7848669445335044


In [24]:
df = df_train.copy()
df = df.append(df_test)
df.to_csv('tmp/selected_data_final.csv')
df

,"Flow rate__fft_coefficient__coeff_1__attr_""angle""",Flow rate__length,"Zone10_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone11_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Zone11_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",Zone15_Pressure__count_below_mean,"Zone15_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__fft_aggregated__aggtype_""kurtosis""",...,"Zone31_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone3_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone4_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone5_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone6_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Zone6_Pressure__fft_coefficient__coeff_2__attr_""angle""",Zone6_Pressure__partial_autocorrelation__lag_2,"Zone7_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone7_Pressure__fft_coefficient__coeff_2__attr_""angle""",target
0,-133.156136,949.0,-2.964372e+04,95.686572,59.248009,0.505982,45.611017,568.0,75.028473,5.776310,...,-9.310171e+05,148.489858,-156.469317,-139.481181,102.743651,-170.198348,-0.110227,1.712034,92.717040,0
1,-156.043635,863.0,-1.146507e+06,22.493810,41.514168,-1.069010,113.872216,566.0,50.880973,7.862577,...,-1.793086e+06,126.229099,129.473073,131.994737,22.769477,130.317641,-0.399107,-1.096303,98.660564,1
2,-129.957522,992.0,3.226080e+05,122.991554,56.670234,1.841951,75.079191,648.0,78.834497,5.784401,...,1.614355e+06,80.454668,61.397743,53.669202,127.463038,87.747322,-0.190496,1.056589,78.150238,0
3,-159.279799,867.0,-1.819657e+06,2.819112,37.574585,-1.185652,97.722058,539.0,43.575737,9.189750,...,-8.465515e+05,134.725121,141.749812,143.606770,5.531214,136.339948,-0.368715,-1.208694,99.725616,1
4,-133.498270,991.0,-1.776349e+05,98.944519,58.988249,-0.474574,82.765689,617.0,64.398653,6.897727,...,2.690433e+05,139.196540,130.846438,129.830750,72.038636,133.019809,-0.191307,-0.531102,110.545236,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,-142.679814,937.0,7.482067e+05,103.862143,56.600148,1.839317,30.990656,597.0,80.847058,5.507917,...,-2.127680e+05,112.034548,136.854022,153.799606,136.777807,145.814711,-0.136794,2.046991,96.602649,0
3001,-124.202911,1021.0,5.485390e+05,98.311475,63.327392,3.347951,51.184197,725.0,81.584063,5.707252,...,1.738431e+06,59.650402,23.246407,20.698919,143.415233,63.488272,-0.042376,5.188739,61.201151,0
3002,-127.967504,979.0,1.585652e+06,106.610445,47.947812,0.193662,43.674917,667.0,74.078531,6.001075,...,2.055724e+06,80.849471,70.065834,65.421438,113.357949,78.059757,-0.314073,-0.211791,74.467876,1
3003,-134.856590,955.0,2.071081e+05,98.591404,55.887443,0.307087,56.076922,579.0,75.003324,5.799452,...,-7.995533e+05,139.537305,-172.987457,-160.031434,96.698425,156.721020,-0.107051,1.298789,96.114856,0


### 6. Unify all data

In [25]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
automatic_scoring_cv(X, y)

0.8572734189966165